# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
#!pip install folium
#!pip install gmaps
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import folium
import os
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(100)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...,...
95,95,kavaratti,10.5669,72.6420,27.12,78,100,7.00,IN,1666108292
96,96,zhangye,38.9342,100.4517,11.11,16,0,1.67,CN,1666108292
97,97,shenjiamen,29.9576,122.2980,16.43,59,1,7.40,CN,1666108292
98,98,itarema,-2.9248,-39.9167,31.07,54,14,9.48,BR,1666108293


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display
# Define Latitude and Longitude Values
Lat = 29.9576
Lng = 122.2980

# Configure the map plot
m = folium.Map(location=[Lat, Lng], zoom_start=4)

# Add markers for each city with the size of the point representing humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  
        color='black',
        fill=True,
        fill_color=['green']
    ).add_to(m)

# Display the map
m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# Define your ideal weather condition criteria
ideal_conditions = (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
# Filter the DataFrame based on the ideal conditions
ideal_cities_df = city_data_df[ideal_conditions]
# Display the filtered DataFrame
print(ideal_cities_df)

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print(ideal_cities_df.head())

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
45        45           kapaa  22.0752 -159.3190     22.99        84   
51        51            hilo  19.7297 -155.0900     26.27        83   
63        63           banda  25.4833   80.3333     24.62        52   
81        81   makakilo city  21.3469 -158.0858     21.66        81   
90        90    saint george  37.1041 -113.5841     13.33        44   
96        96         zhangye  38.9342  100.4517     11.11        16   
115      115      cedar city  37.6775 -113.0619     14.11        49   
121      121         lichuan  30.3000  108.8500      9.72        77   
128      128   half moon bay  37.4636 -122.4286     19.71        86   
141      141         brenham  30.1669  -96.3977     14.47        40   
152      152         kahului  20.8947 -156.4700     23.80        60   
159      159       geraldton -28.7667  114.6000     13.23        88   
197      197             gat  31.6100   34.7642     24.38       100   
211   

### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
5,ushuaia,AR,-54.8000,-68.3000,81,
6,albany,US,42.6001,-73.9662,74,
7,carnarvon,AU,-24.8667,113.6333,76,
8,sisimiut,GL,66.9395,-53.6735,98,
9,saint-pierre,RE,-21.3393,55.4781,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = { "radius": radius,
    "type": "hotel",  
    "limit": 1,     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng'] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}" 
    params["bias"] =  f"proximity:{lng},{lat}" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response_json = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

asau - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
pevek - nearest hotel: No hotel found
husavik - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
zyryanka - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
baykit - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
meulaboh - nearest hotel: No hotel found
nakamura - nearest hotel: No hotel found
simao - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
kodinsk - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
glasgow - nearest hotel: No hotel found
ladario - nearest hotel: No hotel found
sao teotonio - nearest hotel: No hotel found
puri - nearest hotel: No hotel found
yumbe - nearest hotel: No hotel found
pangkalanbuun - nearest hotel: No hotel found
hay river - nearest hotel: No hotel found
verkhniye achaluki - nearest hot

carlyle - nearest hotel: No hotel found
kayerkan - nearest hotel: No hotel found
moron - nearest hotel: No hotel found
paamiut - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
seoul - nearest hotel: No hotel found
woodward - nearest hotel: No hotel found
georgiyevskoye - nearest hotel: No hotel found
mayo - nearest hotel: No hotel found
kysyl-syr - nearest hotel: No hotel found
tashara - nearest hotel: No hotel found
acajutla - nearest hotel: No hotel found
capoterra - nearest hotel: No hotel found
muisne - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
maniitsoq - nearest hotel: No hotel found
kalemie - nearest hotel: No hotel found
tagusao - nearest hotel: No hotel found
mao - nearest hotel: No hotel found
mahibadhoo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
panuco - nearest hotel: No hotel found
hervey bay - nearest hotel: No hotel found
fayaoue - nearest hotel: No hotel found
stolac - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No hotel found
3,jamestown,US,42.0970,-79.2353,77,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Define a function to create hover content for each city marker
def create_hover_content(city, country, hotel_name):
    hover_content = f"City: {city}, Country: {country}, Hotel: {hotel_name}"
    return hover_content

# Create a list to store hover content for each city
hover_content_list = []

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row['City']
    country = row['Country']
    hotel_name = row['Hotel Name']
    # Create hover content for the current city
    hover_content = create_hover_content(city, country, hotel_name)
    hover_content_list.append(hover_content)

# Configure the map plot
fig = gmaps.figure()

# Define the coordinates for each city
coordinates = list(zip(hotel_df['Lat'], hotel_df['Lng']))

# Create markers for each city on the map
markers = gmaps.marker_layer(coordinates, info_box_content=hover_content_list)

# Add the markers to the map
fig.add_layer(markers)

# Display the map
fig

AttributeError: module 'collections' has no attribute 'Iterable'